In [1]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta
from SML.training_pipeline_new import model_dir
import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/693399/jobs/named/tesla_stocks_fv_1_create_fv_td_09052024125854/executions
2024-05-09 15:00:07,187 WARNING: VersionWarning: Incremented version to `14`.

2024-05-09 15:00:08,294 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 0.5285
2024-05-09 15:00:12,985 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/d

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/291253 elapsed<00:00 remaining<?

2024-05-09 15:00:28,565 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



Uploading: 0.000%|          | 0/43 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/561 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/693399/models/stock_pred_model/15


True

In [2]:
api_key = os.environ.get('hopsworks_api')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/693399
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
start_date = datetime.now() - timedelta(hours=48)
print(start_date.strftime("%Y-%m-%d"))

2024-05-07


In [4]:
end_date = datetime.now() - timedelta(hours=24)
print(end_date.strftime("%Y-%m-%d"))

2024-05-08


In [5]:
feature_view = fs.get_feature_view('tesla_stocks_fv', 3)
feature_view.init_batch_scoring(training_dataset_version=2)

In [6]:
print(feature_view.get_batch_query())

WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`date` `date`, `fg1`.`ticker` `ticker`, `fg1`.`ticker` `join_pk_ticker`, `fg1`.`date` `join_evt_date`, `fg0`.`sentiment` `sentiment`, RANK() OVER (PARTITION BY `fg1`.`ticker`, `fg1`.`date` ORDER BY `fg0`.`date` DESC) pit_rank_hopsworks
FROM `klittefr_featurestore`.`tesla_stock_3` `fg1`
INNER JOIN `klittefr_featurestore`.`news_sentiment_updated_3` `fg0` ON `fg1`.`ticker` = `fg0`.`ticker` AND `fg1`.`date` >= `fg0`.`date`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`date` `date`, `right_fg0`.`ticker` `ticker`, `right_fg0`.`sentiment` `sentiment`
FROM right_fg0)


In [7]:
# we had problems fetching the data from fv with get_batch_data function, tried everything and it just doesnt work 
tesla_df_b = feature_view.get_batch_data(start_time=start_date, end_time=end_date)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.29s) 


In [8]:
features_df = tesla_df_b.iloc[:,2]

In [9]:
tesla_df_b.head()

,date,ticker,sentiment
0,2024-05-08 00:00:00+00:00,TSLA,0.010694


In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
#OneHotEncoding the tesla_df_b column 'ticker'

tickers = tesla_df_b[['ticker']]

# Initializing OneHotEncoder
encoder = OneHotEncoder()

# Fitting and transforming the 'ticker' column
ticker_encoded_test = encoder.fit_transform(tickers)

# Converting the encoded column into a DataFrame
ticker_encoded_df_test = pd.DataFrame(ticker_encoded_test.toarray(), columns=encoder.get_feature_names_out(['ticker']))

# Concatenating the encoded DataFrame with the original DataFrame
tesla_df_b = pd.concat([tesla_df_b, ticker_encoded_df_test], axis=1)

# Dropping the original 'ticker' column
tesla_df_b.drop('ticker', axis=1, inplace=True)

In [12]:
# As X_train['date'] column exists and is in datetime format, we're converting it
tesla_df_b['year'] = tesla_df_b['date'].dt.year
tesla_df_b['month'] = tesla_df_b['date'].dt.month
tesla_df_b['day'] = tesla_df_b['date'].dt.day

# Dropping the original date column
tesla_df_b.drop(columns=['date'], inplace=True)

# Converting dataframe to numpy array
tesla_df_b_array = tesla_df_b.to_numpy()

# Reshaping the array to have a shape suitable for LSTM
tesla_df_b_array = np.expand_dims(tesla_df_b_array, axis=1)

2024-05-09 15:00:50,625 WARNING: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)



In [13]:
import joblib

the_model = mr.get_model("stock_pred_model", version=9)
model_dir = the_model.download()

model = joblib.load(model_dir + "/stock_prediction_model.pkl")

In [14]:
predictions = model.predict(tesla_df_b_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 853ms/step


In [15]:
predictions 

array([[0.18933737]], dtype=float32)

In [16]:
tesla_df_b['predictions'] = predictions.tolist()

In [17]:
# Assuming you have 'year', 'month', and 'day' columns in your DataFrame
tesla_df_b['date'] = pd.to_datetime(tesla_df_b[['year', 'month', 'day']])

# Now you can drop the 'year', 'month', and 'day' columns if you want
tesla_df_b.drop(columns=['year', 'month', 'day'], inplace=True)

In [18]:
tesla_df_b['date'] = pd.to_datetime(tesla_df_b['date'])

In [19]:
tesla_df_b

,sentiment,ticker_TSLA,predictions,date
0,0.010694,1.0,[0.1893373727798462],2024-05-08


In [24]:
tesla_df_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sentiment    1 non-null      float64       
 1   ticker_TSLA  1 non-null      float64       
 2   predictions  1 non-null      object        
 3   date         1 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 164.0+ bytes


In [25]:
results_fg = fs.get_or_create_feature_group(
    name= 'Stock prediction results',
    version = 1,
    description = 'Predction of TSLA open stock price',
    primary_key = ['ticker_TSLA'],
    event_time = ['date'],
    online_enabled = False,
)

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/693399/featurestores/689222/featuregroups/Stock%20prediction%20results). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":120004,"usrMsg":"HTTP 404 Not Found","errorMsg":"Web application exception occurred"}', error code: 120004, error msg: Web application exception occurred, user msg: HTTP 404 Not Found